# Speech Processing - Instituto Superior Técnico
### Laboratory Assignment 2 - Spoken Language Indentification challenge

The second laboratory assignment of the course is designed to simulate a **spoken language identification** challenge. In this challenge, partipants (a.k.a students enrolled in the course) receive a train, development and evaluation (blind) data set, and a simple (and weak) baseline system for the task at hand: closed-set identification of the spoken language in a given audio file out of a set of six target langauges: Basque,  Catalan,  English,  Galician,  Portuguese and  Spanish.

The **goal** for each participant is to develop/build the best spoken language identification system. To this end, participants are encouraged to modify this baseline, incorporate any other techniques and in general explore any approach that permit  improving their results.

During the first week (Part 1), students are expected to:
- Run and understand the main components of the baseline.
- Propose and develop simple modifications to the baseline feature extraction process.
- Propose and develop simple modifications to the baseline GMM language models.
- Evaluate the models on the development partition.

During the second week (Part 2), students are expected to:
- Propose and develop other *classical* modifications to any component of the processing pipeline (openSMILE features, segment-based features, SVM classifiers, MLP/CNN classifiers, etc.)
- Run and understand the second part of Notebook that explores a pre-trained model.
- Propose and develop more recent advanced approaches, including x-vectors.
- Evaluate the models on the development partition.
- Obtain predictions for the blind test partition and prepare the submission.

The challenge distinguishes two different tracks or evaluation conditions:
- Track 1 - Participants are not allowed to use any kind of pre-trained model (such as x-vectors).
- Track 2 - Participants are allowed to use anything.


## About the data

The data consists of mono audio files sampled at 16 kHz all of them containing speech of only one of the following target languages:
```python 
LANGUAGES = ('Basque',  'Catalan',  'English',  'Galician',  'Portuguese',  'Spanish')
```

The dataset is organized in 4 partitions:
- `'train'`: This is the full training set, consisting of 3060 clean audio samples correponding to speech segments of TV broadcast shows. (**ATENTION**: Do not use this dataset for training your models, unless your system is very fast or if you want to build your final model. It can be slow)
- `'train100'`: This is a subset of the full training set that consists of 100 audio files per target language (**RECOMMENDATION**: Use this partition in your quick experiments, to more rapidly validate alternatives)
- `'dev'`: This is the development set. It contains audio extracted from YouTube. You will typically use this to validate the quality of your model.
- `'evl'`: This is the evaluation set. It contains audio extracted from YouTube. You don't have the groud-truth for this set. You are expected to produce it and submit it.

The data used in this challenge is a subset of the KALAKA-3 database: https://aclanthology.org/L14-1576/

The  difference is that only the clean train audio segments and the Plenty Closed evaluation condition have been considered. 

## Before starting
The following conditions are neecessary to run correctly this notebook:

*   All modules included in the requirements file need to be 
installed in the Python environment.
*   The module `pf_tools` needs to be accessible (if you are using Google Colab, you will need to copy the `pf_tools.py` every time you start a new session).

In [1]:
from pf_tools import CheckThisCell

## How can you download (and process) the data

The first thing we have to do is to set our working directory. If you are using Google Colab, you  probably want to mount Google Drive to keep persistent information, such as data, features and models:

In [2]:
#raise CheckThisCell ## <---- Remove this after completeing/checking this cell
import os 
CWD = os.getcwd() + '' # Change this variable to your working directory to store data, features and models
CWD

'/home/av/IST/slp-labs/LAB2'

The class `Kalaka` permits downloading, transforming and storing the different data partitions. Each `Kalaka` instance can be used to iterate over all the samples of the partition. It can also be used in combination with pytorch dataloader to read batches of data to train neural networks with pytorch. For instance, consider the following piece of code:


In [3]:
import numpy as np
from pf_tools import Kalaka
import librosa 

def audio_transform(filename):
    y, _ = librosa.load(filename, sr=16000, mono=True)
    return y.reshape(-1,1)
    
#trainkalaka = Kalaka(CWD, 'train100', transform_id='raw', audio_transform=audio_transform)


This will first download and uncompress the .tar.gz file containing all the necessary data of the `'train100'` partition, that is, the audio files that are stored to disk (in CWD/train100/audio/) and key file (CWD/train100/key.lst). Then, the audio transformation `'transform'` will be applied to each file and the result stored to disk CWD/train100/raw/. 

**Audio transformations** receive a filename of an audio file and returns an array of dimensions (NxD), in which N is the time dimension and D the dimension of the feature vector. In this simple case D is 1 because the transform is just returning the raw audio signal.

The `Kalaka` class permits chunking the output of the audio transformation (of size NxD) in chunks of CxD size. The chunking operation divides the orignal sample, in multiple smaller samples with a configurable chunk size and hop length. These chunks can be further transformed and stored as individual feature files. For instance:

In [4]:
trainkalaka = Kalaka(CWD, 'train100', 
                     transform_id='chunks', 
                     audio_transform=audio_transform, 
                     chunk_size=4*16000, 
                     chunk_hop=2*16000)

/home/av/IST/slp-labs/LAB2/pf_tools.py:89: UserWarning: The feature directory already exists, and no new feature extraction will be performed.
  warnings.warn("The feature directory already exists, and no new feature extraction will be performed.")


This will download and uncompress the partition data, only if was not already done before. Then, as previously, the simple tranform that returns the waveform is applied to each audio file. After this, the resulting array of dimension Nx1, in which N=16000xduration_in_seconds, is split in continuous chunks of length 64000 (that is, 4 seconds) with chunk hop of 2 seconds. Each one of these chunks of 4 seconds is stored and will be accessed whenever we iterate the dataset. 

Adittionally, the optional argument `chunk_transform` pertmits defining a transformation to be applied to each chunk before storing to disk. It can be any function that receives an array of size CxD and returns an array HxW, in which H is the *new time dimension*. For instance, the following example takes the audio segments of 64000x1, computes the mean and variance every 0.1 sec (1600 samples) and returns a feature vector of size 40x2.

In [5]:
def chunk_transform(x):
    x = x.reshape(-1,1600)
    return np.concatenate((x.mean(axis=1, keepdims=True), x.std(axis=1, keepdims=True)),axis=1)

trainkalaka = Kalaka(CWD, 'train100', 
                     transform_id='chunks_mv', 
                     audio_transform=audio_transform, 
                     chunk_size=4*16000, 
                     chunk_hop=2*16000, 
                     chunk_transform=chunk_transform)



Notice that, while the above example is probably useless as an effective feature extraction method, the proper combination of audio and chunk transformations is expected to permit quite flexible feature extraction that (hopefully) can match the needs of almost any training setting. 

Once we have instanciated a Kalaka dataset, it can be iterated to have access to each processed sample, for instance: 

In [6]:
import time
start = time.time()
for i, sample in enumerate(trainkalaka):
    data, label, basename = sample # array, int, str
    if i % 1000 == 0:
        print(i, data.shape, label, basename)

print(f'Finished reading all data in {time.time() - start}')

0 (40, 2) 6 0006ebda
1000 (40, 2) 1 11a3790f
2000 (40, 2) 4 271621da
3000 (40, 2) 5 3b9d2c2f
4000 (40, 2) 1 4d10f8f2
5000 (40, 2) 6 5fb949a1
6000 (40, 2) 6 70d28e30
7000 (40, 2) 6 7d5515f0
8000 (40, 2) 3 937623c2
9000 (40, 2) 5 a4a7491b
10000 (40, 2) 5 bc3f6e51
11000 (40, 2) 3 cd82bcca
12000 (40, 2) 1 df0b27f2
13000 (40, 2) 3 f3ba1f4d
Finished reading all data in 10.039682149887085


Now you can use the `Kalaka` class to check the  number of files and size (in minutes) of the training set for each target language. You cankKeep these numbers to include in your system description paper:

Notice that the `Kalaka` class extends the `torch.utils.data.Dataset` and it can be used in combination with a Pytorch DataLoader to read data in batches:

In [7]:
import torch 

trainkalaka = Kalaka(CWD, 'train100', 
                     transform_id='chunks_mv', 
                     audio_transform=audio_transform, 
                     chunk_size=4*16000, 
                     chunk_hop=2*16000, 
                     chunk_transform=chunk_transform)

dataloader = torch.utils.data.DataLoader(
        dataset=trainkalaka,
        batch_size=10,
        shuffle=True
)

start = time.time()
for i, batch in enumerate(dataloader):
    data, label, basename = batch
    if i % 100 == 0:
        print(data.shape, label.shape, len(basename))

print(f'Finished reading all data in {time.time() - start}')


torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
torch.Size([10, 40, 2]) torch.Size([10]) 10
Finished reading all data in 3.8828232288360596


Finally, remind that you can change anything you want. This includes the Kalaka class. Eventually, you can decide not using it at all and loading the data in some alternative way.  You can inspect the class to find the URLs for downloading the datasets. It is up to you! 

Before moving to the next stage, you probably want to delete the folders containing the dummy features that you just generated.

# PART 1 - The baseline (Track 1)
The baseline consists of MFCC feature extraction  (based on the `librosa` module) with SDC computation and VAD removal, followed by GMMs of 64 dimensions for each language (using the `sklearn` module). The rest of this notebook contains the guide and code cells (some of them partially incomplete) that permit implementing this baseline and score it on the development set. Read carefully the Markdown information, but also the comments inside the code cells (they provide useful information and hints), and also the code itself. The better you understand it, the easier will be modyfing it.

## Initialization and importing modules

In [24]:
import importlib

try:
    # importlib.import_module('opensmile')
    import opensmile
    print('OK')

except ImportError:
    # print('Package not installed')
    !pip install opensmile

OK


In [122]:
import os
from pf_tools import Kalaka
import librosa
import numpy as np
import torch
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

import time
import random
import datetime
from pathlib import Path
import pickle
import csv 
from tqdm import tqdm
import opensmile #feature extraction
import importlib #reload packages

opensmile=importlib.reload(opensmile)

GLOBAL_SEED = 35731

np.random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)


LANGUAGES = ('Basque',  'Catalan',  'English',  'Galician',  'Portuguese',  'Spanish')
LANG2ID = {'Basque':1, 'Catalan':2, 'English':3, 'Galician':4, 'Portuguese':5, 'Spanish':6}
ID2LANG = dict((LANG2ID[k],k) for k in LANG2ID)

## The baseline feature extraction module
The next function extracts MFCCs, but there are plenty of things that can be improved. You are free to change anything you want, including the number of formal parameters, the number of returned expressions, etc. 

In [10]:
# Read carefully this function and understand it

def feat_extract(filename, orig_sr=16000, mono=True, n_mfcc = 13, remove_c0=False, delta_order=0, 
                 apply_sdc=False, apply_vad=False, apply_cmvn=False):
    
    sr=16000
    n_mels = 40
    n_fft = 512 
    hop_length = 160
    fmin = 50
    fmax = 7800
    
    if apply_sdc and (delta_order > 0):
        raise ValueError("Applying SDC and delta > 0 is not compatible")

    
    # Load audio wav into numpy array
    y, _ = librosa.load(filename, sr=orig_sr, mono=mono)
    
    # Resample in case it's needed
    if orig_sr != sr:
        y = librosa.resample(y, orig_sr=orig_sr,target_sr=sr)
    
        
    ## OPTIONAL ADDIDITIONAL STAGES - LAB WORK
    # 1 - PREPROCESSING - Typical preprocessing may include normalization of audio (mean removal), 
    #                       but also speech enhancement and others more complex
    
    # Extract MFFCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, 
                                n_mfcc=n_mfcc, n_mels=n_mels, 
                                hop_length=hop_length, 
                                fmin=fmin, fmax=fmax, htk=False).T
    
    ## OPTIONAL ADDIDITIONAL STAGES - LAB WORK

    # 2 Compute deltas --> Hint you can use librosa (order size may be a parameter?)
    if delta_order > 0:
        mfcc = compute_delta(mfcc,delta_order=delta_order)
    
    # 3 SDC --> Hint: You can build this using deltas (of extented context)  
    if apply_sdc:
        mfcc = compute_sdc(mfcc)
    
    # 4 COMPUTE VAD --> Hint: You can use any vad (theshold energy, something avaialble in the net, a biGaussian model...).
    #                         Coeff0 is highly related with Energy and sometimes it is removed
    #                   ATTENTION: Using a VAD may have a significant impact  
    if apply_vad:
        mfcc, _ = compute_vad(mfcc)
             
        
    # 5 APPLY CMVN --> ATTENTION: Using normalization may have a significant impact
    if apply_cmvn:
        mfcc = compute_cmvn(mfcc)
        

    return mfcc, y


Try to define some or all of the following steps to improve your feature extraction pipeline 

In [11]:
# Numpy arrays have methods to compute mean and variance, so this one should be really easy
# raise CheckThisCell ## <---- Remove this after completeing/checking this cell

def compute_cmvn(features):
    #return features/np.linalg.norm(features)
    return (features-np.mean(features))/np.std(features)

In [12]:
# librosa contains functions to compute deltas
# raise CheckThisCell ## <---- Remove this after completeing/checking this cell

def compute_delta(features, win=3, delta_order=2, keep_static=True):
    delta = librosa.feature.delta(
        features.T,
        width=win,order=delta_order
    ).T
    return np.hstack((features,delta))

In [13]:
# Compute deltas and then select previous and next deltas with fixed intervals to cocatentate. 
# You may need to code a bit here or dinf some function that helps with this
# raise CheckThisCell ## <---- Remove this after completeing/checking this cell

def compute_sdc(features, P=3, D=3, K=2, keep_static=True):
    delta = compute_delta(
        features.T,
        win=D, delta_order=1)[:,np.shape(features)[1]:]
    
    sdc = np.empty([np.shape(features)[0],np.shape(features)[1]*K])
    marker = np.zeros(features.shape[0], dtype='bool')
    for j in range(K):
        marker[D + j * P] = True
    for i in range(len(features)):
        sdc[i, :] = delta[marker, :].reshape(1, -1)
        marker = np.roll(marker, 1)
    return np.hstack((features,sdc))

In [14]:
# You can think of several strattegies to compute VAD, simple ones based on energy and a threshold, or maybe some more 
# ellaborated ones, like training a GMM with 2 mixtures with the Energy. 
# In addition to the features without some frames, this function may return a sequence of 0s and 1s that helps you to validate the method.
# raise CheckThisCell ## <---- Remove this after completeing/checking this cell

# Voice Activity Detection - Zero Crossing Rate
def zero_crossing_rate(x):
    return np.sum(np.abs(np.diff(np.sign(x)))/2)/len(x)

def compute_vad(features, energy=None, y=None):
    features_aux=features[:,0:int(np.shape(features)[1]/2)]
    zcr=np.empty(np.shape(features)[1])
    for i in range(len(zcr)):
        zcr[i]=zero_crossing_rate(features[:,i])
    GM=GaussianMixture(n_components=2)
    frame_class=GM.fit_predict(zcr.reshape(-1,1))
    vad=np.empty(np.shape(features))
    for i in range(len(zcr)):
        vad[:,i]=frame_class[i]
    return np.hstack((features,vad)),frame_class

In [15]:
def opensmile_feats(signal,sr):
    smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.GeMAPSv01b ,
            feature_level=opensmile.FeatureLevel.Functionals,
        )
    opensmile_feats=smile.process_signal(signal,sr)
    return np.hstack(opensmile_feats.values)

You can test in an isolated audio file and inspect the dimensions, verify that your code is doing what is expected, inspect and visualize the data using some of the lessons learnt in LAB1. Also, don't forget to listen some of the examples!!

In [16]:
# raise CheckThisCell ## <---- Remove this after completeing/checking this cell
mfcc, _=feat_extract(f'{CWD}/train100/audio/0006ebda.wav', apply_cmvn=True, apply_sdc=False,apply_vad=False)
mfcc_dd, _ = feat_extract(f'{CWD}/train100/audio/0006ebda.wav', apply_cmvn=False, delta_order=2)
mfcc_d, _ = feat_extract(f'{CWD}/train100/audio/0006ebda.wav', apply_cmvn=False, delta_order=1)
mfcc, y = feat_extract(f'{CWD}/train100/audio/0006ebda.wav', apply_cmvn=False)


To run the data processing stage for the train100 partition we will simply instanciate the Kalaka class as mentioned previosly. Take a sit because it can take a bit:

In [17]:
#Transformation  Combinations attempted    BEST: mfcc_delta1
transform = {
    'mfcc_sdc_vad_chunk_300_400' : {
        'audio_transform': lambda x : feat_extract(x, orig_sr=16000, mono=True, n_mfcc = 7, apply_sdc=False, apply_vad=False, apply_cmvn=True)[0],
        'chunk_transform': None,
        'chunk_size': 300,
        'chunk_hop':300
    },

    'mfcc_delta1_vad' : {
        'audio_transform': lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=1, n_mfcc = 13, apply_sdc=False, apply_vad=True, apply_cmvn=True)[0],
        'chunk_transform': None,
        'chunk_size': 300,
        'chunk_hop':300
    },

    'mfcc_sdc' : {
        'audio_transform': lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=0, n_mfcc = 13, apply_sdc=True, apply_vad=False, apply_cmvn=True)[0],
        'chunk_transform': None,
        'chunk_size': 300,
        'chunk_hop':300
    },

    'mfcc_delta1' : {
        'audio_transform': lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=1, n_mfcc = 13, apply_sdc=False, apply_vad=False, apply_cmvn=True)[0],
        'chunk_transform': None,
        'chunk_size': 300,
        'chunk_hop':300
    },

    'mfcc' : {
        'audio_transform': lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=0, n_mfcc = 7, apply_sdc=False, apply_vad=False, apply_cmvn=False)[0],
        'chunk_transform': None,
        'chunk_size': 300,
        'chunk_hop':300
    },
}

trainset = 'train100'
transform_id = 'mfcc'

""" 
trainkalaka = Kalaka(CWD, trainset, 
                 transform_id=transform_id, 
                 audio_transform=transform[transform_id]['audio_transform'], 
                 chunk_transform=transform[transform_id]['chunk_transform'],
                 chunk_size=transform[transform_id]['chunk_size'], 
                 chunk_hop=transform[transform_id]['chunk_hop']
                )
"""

" \ntrainkalaka = Kalaka(CWD, trainset, \n                 transform_id=transform_id, \n                 audio_transform=transform[transform_id]['audio_transform'], \n                 chunk_transform=transform[transform_id]['chunk_transform'],\n                 chunk_size=transform[transform_id]['chunk_size'], \n                 chunk_hop=transform[transform_id]['chunk_hop']\n                )\n"

Check your current folder, many things happened!! 

Notice that if you instanciate again the Kalaka class for the 'train100' partition, the data will not be downloaded again. 
Additionally, if there is already a folder with the name `transform_id`, feature extraction will not run again. You need to delete from your filesystem the folder with the features if you want to run again the feature extraction (using the same identifier) or , alternatively, you can change the identifier. Be careful because you can easily increase the amount of data generated. If you try a feature extraction method that provides bad results, you probably don't want to keep the features in disk.

## The baseline spoken language models
The baseline model is extremely simple: we'll train an individual GMM model for each language on top of the features that we just extracted. Later, in prediction time, given a test audio sample, we'll compute the loglikelihood obtained with each GMM model and select as the identified language the one whose model gives the highest likelihood. Let's go for it!!


In [18]:
# from sklearn.model_selection import GridSearchCV # Not useful for GMM's

transform_id = 'mfcc'
parameters = {
    # 'transform_id': ['feat', 'mfcc'],
    'audio_transform': [
        lambda x : feat_extract(x, orig_sr=16000, mono=True, n_mfcc = 7, apply_sdc=False, apply_vad=False, apply_cmvn=True)[0], # mfcc_cmvn
        lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=1, n_mfcc = 13, apply_sdc=False, apply_vad=True, apply_cmvn=True)[0], # mfcc_delta1_vad_cmvn
        lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=0, n_mfcc = 13, apply_sdc=True, apply_vad=False, apply_cmvn=True)[0], # mfcc_sdc_cmvn
        lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=1, n_mfcc = 13, apply_sdc=False, apply_vad=False, apply_cmvn=True)[0], # mfcc_delta1_cmvn
        lambda x : feat_extract(x, orig_sr=16000, mono=True,delta_order=0, n_mfcc = 7, apply_sdc=False, apply_vad=False, apply_cmvn=False)[0], # mfcc
    ],
    'chunk_size': [1024, 2048],
    'chunk_hop': [512, 1024],

    'n_gauss': [8, 16, 32]
}

In [106]:
import itertools

def parameter_combinations(param_grid):
    keys = param_grid.keys()
    values = param_grid.values()
    combinations = list(itertools.product(*values))
    return [dict(zip(keys, combination)) for combination in combinations]

def concat_kalaka(trainkalaka):
    # print('concatenating Kalaka')
    train_data = []
    train_labels = []
    for data, label, basename in trainkalaka:
            train_data.append(data)
            train_labels.append(np.full(data.shape[0], label)) 

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    return train_data, train_labels

from sklearn.metrics import accuracy_score

def eval_gmm(gmm, kalaka):

    LANGUAGES = ('Basque',  'Catalan',  'English',  'Galician',  'Portuguese',  'Spanish')
    LANG2ID = {'Basque':1, 'Catalan':2, 'English':3, 'Galician':4, 'Portuguese':5, 'Spanish':6}
    ID2LANG = dict((LANG2ID[k],k) for k in LANG2ID)

    dev_data = {}
    for data, label, basename in kalaka:
        if basename not in dev_data:
                dev_data[basename] = {'data':[], 'label':label}

        dev_data[basename]['data'].append(data)

## We concatenate all the frames belonging to the same filename
    for basename in dev_data:
        dev_data[basename]['data'] = np.concatenate(dev_data[basename]['data'])
    
    # return dev_data[basename]

    results = {}
    results['ref'] =  np.empty(len(dev_data), dtype=np.int32)
    results['hyp'] =  np.empty(len(dev_data), dtype=np.int32)
    results['llhs'] = np.empty((len(dev_data), len(LANGUAGES)), dtype=np.float64)
    results['fileids'] = list()
    
    for i, fileid in tqdm(enumerate(sorted(dev_data)), total=len(dev_data)):
        data = dev_data[fileid]['data']  # the features
        
        results['fileids'].append(fileid)     #fileid

        # obtain the log-likelihood score for each model and store
        results['llhs'][i,:] = np.array([gmm[lang].score(data) for lang in LANGUAGES])

        # store the reference.
        results['ref'][i] = (dev_data[fileid]['label']) #reference

        # Obtain the maximum likelihood languge estimation
        ix = np.argmax(results['llhs'][i,:])
        results['hyp'][i] = LANG2ID[LANGUAGES[ix]]
    
    return accuracy_score(results['ref'], results['hyp'])

In [ ]:
combinations = parameter_combinations(parameters)

best_parameters = {
    
    # 'transform_id': None,
    'audio_transform': None,
    'chunk_size': None,
    'chunk_hop': None,
    'n_gauss': None
}
best_score = -float('inf')
best_model = 0
all_acc = []
all_traintime = []

for i, c in enumerate(combinations):
    print('\nCombination', i+1)
    parms = list(c.values())
    '''
    parms[0] = audio_transform (lambda)
    parms[1] = chunk_size
    parms[2] = chunk_hop
    parms[3] = n_gauss
    '''
    start = time.time()
    trainset = 'train100'
    trainkalaka = Kalaka(
        CWD, trainset,
        transform_id='mfcc',
        audio_transform=parms[0],
        chunk_transform=None,
        chunk_size=parms[1],
        chunk_hop=parms[2]
    )

    # print('Training model')
    train_data, train_labels = concat_kalaka(trainkalaka)

    GMM = {}
    for l, lang in enumerate(LANGUAGES):
        GMM[lang] = GaussianMixture(
            n_components = parms[3],
            covariance_type='diag',
            max_iter=15, n_init=1,
            init_params='kmeans',
            verbose=2, verbose_interval=1
        )
        GMM[lang].fit(train_data[train_labels==LANG2ID[lang]])
    all_traintime.append(time.time() - start)

    # score = GMM.score(train_data)
    acc = eval_gmm(GMM, trainkalaka)
    print(f'Accuracy Score = {acc}\n')

    all_acc.append(acc)

    if acc > best_score:
        best_score = acc
        best_model = i+1
        best_parameters = c.copy()
        best_GMM = GMM.copy()



print(f'Best Model #{best_model}')
print('Best Parameters:\n', best_parameters)
print('Best Accuracy =', best_score)

In [182]:
model_history = pd.DataFrame({
    'model': list(range(1, len(combinations[0:3])+1)),
    'parms': combinations[0:3],
    'train_time': all_traintime,
    'acc_score': all_acc
})
model_history.sort_values(
    by=['acc_score', 'train_time'],
    ascending=[False, True],
    inplace=True
)
model_history.reset_index(drop=True)

best_parameters = model_history['parms'][0]

,model,parms,train_time,acc_score
0,3,{'audio_transform': <function <lambda> at 0x7f...,453.231885,0.671667
1,2,{'audio_transform': <function <lambda> at 0x7f...,228.177402,0.631667
2,1,{'audio_transform': <function <lambda> at 0x7f...,87.186932,0.585000


In [120]:
combinations = parameter_combinations(parameters)

best_parameters = {
    
    # 'transform_id': None,
    'audio_transform': None,
    'chunk_size': None,
    'chunk_hop': None,
    'n_gauss': None
}
best_score = -float('inf')
best_model = 0
# best_score = np.full_like(LANGUAGES, float('-inf'))
# best_model = np.zeros(len(LANGUAGES))

for i, c in enumerate(combinations):
    print('Combination', i+1)
    parms = list(c.values())
    '''
    parms[0] = audio_transform (lambda)
    parms[1] = chunk_size
    parms[2] = chunk_hop
    parms[3] = n_gauss
    '''

    trainset = 'train100'
    trainkalaka = Kalaka(
        CWD, trainset,
        transform_id='mfcc',
        audio_transform=parms[0],
        chunk_transform=None,
        chunk_size=parms[1],
        chunk_hop=parms[2]
    )

    # print('Training model')
    train_data, train_labels = concat_kalaka(trainkalaka)

    GMM = {}
    for l, lang in enumerate(LANGUAGES):
        GMM[lang] = GaussianMixture(
            n_components = parms[3],
            covariance_type='diag',
            max_iter=15, n_init=1,
            init_params='kmeans',
            verbose=2, verbose_interval=1
        )
        GMM[lang].fit(train_data[train_labels==LANG2ID[lang]])

    # score = GMM.score(train_data)
    acc = eval_gmm(GMM, trainkalaka)
    print('Accuracy Score:', acc)

    if acc > best_score:
        best_score = acc
        best_model = i
        best_parameters = c.copy()
        best_GMM = GMM.copy()



print(f'Best Model #{best_model}')
print('Best Parameters:\n', best_parameters)
print('Best Accuracy =', best_score)
        

Combination 1


/home/av/IST/slp-labs/LAB2/pf_tools.py:89: UserWarning: The feature directory already exists, and no new feature extraction will be performed.
  warnings.warn("The feature directory already exists, and no new feature extraction will be performed.")


Initialization 0
  Iteration 1	 time lapse 2.66593s	 ll change inf
  Iteration 2	 time lapse 0.96849s	 ll change 1.60017
  Iteration 3	 time lapse 1.35778s	 ll change 0.02191
  Iteration 4	 time lapse 0.89399s	 ll change 0.01384
  Iteration 5	 time lapse 0.81778s	 ll change 0.00924
  Iteration 6	 time lapse 1.13438s	 ll change 0.00637
  Iteration 7	 time lapse 0.97279s	 ll change 0.00459
  Iteration 8	 time lapse 0.84951s	 ll change 0.00348
  Iteration 9	 time lapse 1.26912s	 ll change 0.00277
  Iteration 10	 time lapse 0.93575s	 ll change 0.00229
  Iteration 11	 time lapse 0.89245s	 ll change 0.00195
  Iteration 12	 time lapse 0.84302s	 ll change 0.00167
  Iteration 13	 time lapse 0.77300s	 ll change 0.00144
  Iteration 14	 time lapse 0.78005s	 ll change 0.00124
  Iteration 15	 time lapse 0.86523s	 ll change 0.00106
Initialization converged: False	 time lapse 16.01935s	 ll -27.27245


/home/av/.local/lib/python3.10/site-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


Initialization 0
  Iteration 1	 time lapse 2.29687s	 ll change inf
  Iteration 2	 time lapse 0.78391s	 ll change 1.30890
  Iteration 3	 time lapse 0.82942s	 ll change 0.01549
  Iteration 4	 time lapse 0.86444s	 ll change 0.01023
  Iteration 5	 time lapse 0.94776s	 ll change 0.00802
  Iteration 6	 time lapse 0.95712s	 ll change 0.00678
  Iteration 7	 time lapse 0.87926s	 ll change 0.00594
  Iteration 8	 time lapse 0.91473s	 ll change 0.00529
  Iteration 9	 time lapse 0.81270s	 ll change 0.00470
  Iteration 10	 time lapse 0.95139s	 ll change 0.00415
  Iteration 11	 time lapse 0.99862s	 ll change 0.00366
  Iteration 12	 time lapse 0.94177s	 ll change 0.00322
  Iteration 13	 time lapse 0.98399s	 ll change 0.00285
  Iteration 14	 time lapse 0.94194s	 ll change 0.00254
  Iteration 15	 time lapse 0.77334s	 ll change 0.00229
Initialization converged: False	 time lapse 14.87735s	 ll -27.52054


...

Initialization 0
  Iteration 1	 time lapse 8.23923s	 ll change inf
  Iteration 2	 time lapse 3.46877s	 ll change 1.53026
  Iteration 3	 time lapse 3.41381s	 ll change 0.02084
  Iteration 4	 time lapse 2.76520s	 ll change 0.01313
  Iteration 5	 time lapse 3.39511s	 ll change 0.00966
  Iteration 6	 time lapse 3.12462s	 ll change 0.00752
  Iteration 7	 time lapse 3.01600s	 ll change 0.00609
  Iteration 8	 time lapse 2.72320s	 ll change 0.00509
  Iteration 9	 time lapse 4.12230s	 ll change 0.00434
  Iteration 10	 time lapse 2.86412s	 ll change 0.00374
  Iteration 11	 time lapse 3.24990s	 ll change 0.00323
  Iteration 12	 time lapse 2.67484s	 ll change 0.00281
  Iteration 13	 time lapse 3.38749s	 ll change 0.00247
  Iteration 14	 time lapse 3.06135s	 ll change 0.00220
  Iteration 15	 time lapse 2.83481s	 ll change 0.00199
Initialization converged: False	 time lapse 52.34113s	 ll -26.84063


/home/av/.local/lib/python3.10/site-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


Initialization 0
  Iteration 1	 time lapse 8.75397s	 ll change inf
  Iteration 2	 time lapse 4.02366s	 ll change 0.05944
  Iteration 3	 time lapse 3.11875s	 ll change 0.02011
  Iteration 4	 time lapse 3.06416s	 ll change 0.01140
  Iteration 5	 time lapse 3.27635s	 ll change 0.00780
  Iteration 6	 time lapse 3.35677s	 ll change 0.00577
  Iteration 7	 time lapse 3.17567s	 ll change 0.00446
  Iteration 8	 time lapse 3.52321s	 ll change 0.00359
  Iteration 9	 time lapse 3.22653s	 ll change 0.00301
  Iteration 10	 time lapse 3.08917s	 ll change 0.00263
  Iteration 11	 time lapse 3.19461s	 ll change 0.00238
  Iteration 12	 time lapse 3.47355s	 ll change 0.00225
  Iteration 13	 time lapse 3.29130s	 ll change 0.00221
  Iteration 14	 time lapse 2.92495s	 ll change 0.00224
  Iteration 15	 time lapse 3.42219s	 ll change 0.00232
Initialization converged: False	 time lapse 54.91521s	 ll -27.26465


/home/av/.local/lib/python3.10/site-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
100%|██████████| 600/600 [01:15<00:00,  7.94it/s]

Accuracy Score: 0.6766666666666666
Best Model #50
Best Parameters:
 {'audio_transform': <function <lambda> at 0x7f870cd0c790>, 'chunk_size': 1024, 'chunk_hop': 512, 'n_gauss': 32}
Best Accuracy = 0.6833333333333333


In [ ]:
# IN GMM training each training sample contains more than one frame;
# so we cocatenate all data to have all training datain one array 
# and the corresponding label with same time duration
# actually, here the chunking process is useless, I could have obtained the same without chunking (with slight differences due to trunkation)

#This method cannot be used with opensmile
start = time.time()
train_data = []
train_labels = []
for data, label, basename in trainkalaka:
        train_data.append(data)
        train_labels.append(np.full(data.shape[0], label)) 

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)

print(f'Finished reading all data in {time.time() - start}')

Finished reading all data in 2.7420384883880615


Let's go training. Again, depending of the amount of data used, the model complexity and computational resources of the machine that you're using, this can take a while. So, relax while the computer works for you!

In [ ]:
## TRAIN GMM models (ML)
"""
models = {}
n_gauss = 64
for lang in LANGUAGES:
    models[lang] = GaussianMixture(n_components=n_gauss, covariance_type='diag', max_iter=20, n_init=1, init_params='kmeans', verbose=2, verbose_interval=1)
    
for lang in LANGUAGES:
    print(f'Training model for {lang}')
    models[lang].fit(train_data[train_labels==LANG2ID[lang]])
"""

Now we have two arrays containing the complete training dataset and the corresponging reference labels. Check the sizes, may be have a look to the content of one time instant. Do some checks on the data to be sure that everything is as expected:

## Identification and generation of the predictions file

Now that we  already have trained models, let's predict/identify language in new audio data and test our model!!! 

But first, we need to obtain the development partition and do the feature extraction more or less as previously (using the Kalaka class).

**IMPORTANT WARNING** Make sure to use the exact same feature extraction process as the one used for the train set. Otherwise, your model will be in disagreement with your evaluation data, and very likely, will not work at all.


In [ ]:
# Download and feature extract
# transform_id = 'opensmile'
transform_id = 'mfcc'

# Notice that the chunking is mostly useless both in training and prediction with GMMs: each frame is a sample for which we obtain the probs
devkalaka = Kalaka(
    CWD,'dev', 
    transform_id = transform_id, 
    audio_transform = best_parameters['audio_transform'], 
    chunk_transform = None,
    chunk_size = best_parameters['chunk_size'], 
    chunk_hop = best_parameters['chunk_hop']
)

Now, we can iterate the data and use the models for scoring:

In [ ]:
start = time.time()
dev_data = {}

for data, label, basename in devkalaka:
        if basename not in dev_data:
                dev_data[basename] = {'data':[], 'label':label}
        dev_data[basename]['data'].append(data)

## We concatenate all the frames belonging to the same filename
for basename in dev_data:
        dev_data[basename]['data'] = np.concatenate(dev_data[basename]['data'])

print(f'Finished reading all data in {time.time() - start}')

Finished reading all data in 0.2989621162414551


In [ ]:
start = time.time()

results_dev = {}
results_dev['ref'] =  np.empty(len(dev_data),dtype=np.int32)
results_dev['hyp'] =  np.empty(len(dev_data),dtype=np.int32)
results_dev['llhs'] = np.empty((len(dev_data), len(LANGUAGES)), dtype=np.float64)
results_dev['fileids'] = list()

for i, fileid in tqdm(enumerate(sorted(dev_data)), total=len(dev_data)):
    data = dev_data[fileid]['data']  # the features
    
    results_dev['fileids'].append(fileid)     #fileid

    # obtain the log-likelihood score for each model and store
    results_dev['llhs'][i,:] = np.array([best_GMM[lang].score(data) for lang in LANGUAGES])

    # store the reference. Notice that I only have this for the dev set, not for the eval
    results_dev['ref'][i] = (dev_data[fileid]['label']) #reference

    # Obtain the maximum likelihood languge estimation
    ix = np.argmax(results_dev['llhs'][i,:])
    results_dev['hyp'][i] = LANG2ID[LANGUAGES[ix]]
    
    
print(f'Finished predicting all data in {time.time() - start}')



## Evaluation
After running the previous cells, we obtain two arrays with the reference and hypothesis labels, respectively (we can also reload them in case we need them). We can use these to obtain different evaluation metrics and inspect the performance (and potential problems) of our system. Of course, you will only be able to do this evaluation with the development set, since you don't have access to the eval labels.

You can for instance obtain a classification report summary:

In [ ]:
ref, hyp = results_dev['ref'], results_dev['hyp']
print(classification_report(ref, hyp, target_names=LANGUAGES))

              precision    recall  f1-score   support

      Basque       0.23      0.04      0.07       154
     Catalan       0.17      0.05      0.08       149
     English       0.16      0.75      0.27       150
    Galician       0.20      0.03      0.06       151
  Portuguese       0.07      0.04      0.05       160
     Spanish       0.12      0.03      0.05       153

    accuracy                           0.16       917
   macro avg       0.16      0.16      0.10       917
weighted avg       0.16      0.16      0.09       917



Overall accuracy (this will be the **main metric for system ranking**) Or a confusion matrix:

In [ ]:
print('Accuracy Score =', accuracy_score(ref, hyp))
print('Confusion Matrix =', confusion_matrix(ref, hyp))

0.1559432933478735

and visualize it:

In [ ]:
conf_matrix = confusion_matrix(ref, hyp)

fig, ax = plt.subplots(figsize=(3.0, 3.0))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center')

plt.xlabel('Predictions', fontsize=12)
plt.ylabel('Actuals', fontsize=12)
plt.title(f'Confusion Matrix\n(Accuracy {100*accuracy_score(ref, hyp):.2f})', fontsize=12)
plt.show()

These results are poor. The baseline system is very limited in several aspects (features, time context, generative model, etc.). For instance, the likelihood scores are not normalized. It may happen, that one model  provides slightly higher scores for some reason. The following trick sometimes increases slightly the results:

In [ ]:
llhs_norm = (results_dev['llhs'] - results_dev['llhs'].mean(axis=0))
hyp_norm = np.empty(hyp.shape, like=hyp)
for i in range(len(results_dev['fileids'])):
    ix = np.argmax(llhs_norm[i,:])
    hyp_norm[i] = LANG2ID[LANGUAGES[ix]]


In [ ]:
conf_matrix = confusion_matrix(ref, hyp_norm)

fig, ax = plt.subplots(figsize=(3.0, 3.0))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center')

plt.xlabel('Predictions', fontsize=12)
plt.ylabel('Actuals', fontsize=12)
plt.title(f'Confusion Matrix\n(Accuracy {100*accuracy_score(ref, hyp_norm):.2f})', fontsize=12)
plt.show()

This "trick" is probematic. Can you discuss why? Can you think of an altenative way of doing the same kind of normalization?

# Creating Dataset from opensmile features

In [ ]:
#For opensmile:
train_data = []
train_labels = []
trainkalaka = Kalaka(CWD, 'train100', transform_id='raw', audio_transform=audio_transform)


start = time.time()
train_data = []
train_labels = []
i=0
for data, label, basename in trainkalaka:
    print(i)
    train_data.append(opensmile_feats(data.reshape(np.shape(data)[0]),sr=16000))
    train_labels.append([label]) 
    if i%100==0:
        print(i)
    i=i+1
        
train_data = np.vstack(train_data)
train_labels = np.concatenate(train_labels)
joblib.dump(dev_data,'train100\\opensmile.pkl')


In [ ]:
#For opensmile:
dev_data = []
dev_labels = []
devkalaka = Kalaka(CWD, 'dev', transform_id='raw', audio_transform=audio_transform)


start = time.time()
train_data = []
train_labels = []
i=0
for data, label, basename in devkalaka:
    print(i)
    dev_data.append(opensmile_feats(data.reshape(np.shape(data)[0]),sr=16000))
    dev_labels.append([label]) 
    if i%100==0:
        print(i)
    i=i+1
        
dev_data = np.vstack(dev_data)
dev_labels = np.concatenate(dev_labels)


In [ ]:
#Let's train a GBDT
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
from lightgbm import plot_tree
import sklearn as skl
from sklearn.model_selection import ParameterSampler
import numpy as np
import math
from scipy.stats import uniform

def lgbm_param_sampler(n_iters,seed,device):
    param_grid={'num_iterations': 
                range(30, 200),
        'learning_rate':
            uniform(loc=0.001, scale=1-0.001 ),
        'num_leaves':
            range( 5, 100 ),
        'boosting_type':
            ["goss"],
        'min_data_in_leaf':
            range( 2, 50 ),
        'max_bin':
            range( 3, 100 ),
        'device': 
            [device]}
    param_list = list(ParameterSampler(param_grid, n_iter=n_iters, random_state=np.random.RandomState(seed)))
    return param_list

params=lgbm_param_sampler(50,1213,'cpu')
i=0
for k in params:
    lgbm=LGBMClassifier(**k,objective='multiclass',verbose=-1,verbose_eval=-1)
    lgbm.fit(train_data,train_labels)
    y_hat=lgbm.predict(dev_data)
    print('iter :',i,'--- dev accuracy: ',accuracy_score(y_hat,dev_labels)) #???
    #print(confusion_matrix(dev_labels, y_hat))
    i=i+1

In [ ]:
start = time.time()

results_dev = {}
results_dev['ref'] =  np.empty(len(dev_data),dtype=np.int32)
results_dev['hyp'] =  np.empty(len(dev_data),dtype=np.int32)
results_dev['estimated_prob'] =  np.empty((len(dev_data), len(LANGUAGES)), dtype=np.float64)
results_dev['fileids'] = list()

for i, fileid in tqdm(enumerate(sorted(dev_data)), total=len(dev_data)):
    data = dev_data[fileid]['data']  # the features
    
    results_dev['fileids'].append(fileid)     #fileid

    # obtain the log-likelihood score for each model and store
    results_dev['estimated_prob'][i,:] = np.array([lgbm.predict_proba(data.reshape(1,-1))[0,i] for i in range(len(LANGUAGES))])

    # store the reference. Notice that I only have this for the dev set, not for the eval
    results_dev['ref'][i] = (dev_data[fileid]['label']) #reference

    # Obtain the maximum likelihood languge estimation
    ix = lgbm.predict(data.reshape(1,-1))
    results_dev['hyp'][i] = ix
    
    
print(f'Finished predicting all data in {time.time() - start}')


100%|██████████| 917/917 [00:02<00:00, 306.21it/s]

Finished predicting all data in 2.9981255531311035


## Running the prediction on the evl partition

Once you are happy with your system and the results obtained in the development set, you are ready to generate the predictions on the `'evl'` partition. To do that, you have to follow the same process as for the development partition, but of course, this time you will not be able to obtain performance results because you don't have labels for this partition. 

We start by instantiating the `Kalaka` class for the `'evl'` partition:


In [146]:
transform_id = 'mfcc_sdc_vad_chunk_300_300'

evlkalaka = Kalaka(
    CWD,'evl',
    transform_id=transform_id,
    audio_transform=transform[transform_id]['audio_transform'],
    chunk_transform=transform[transform_id]['chunk_transform'],
    chunk_size=transform[transform_id]['chunk_size'], 
    chunk_hop=transform[transform_id]['chunk_hop']
)


KeyError: 'mfcc_sdc_vad_chunk_300_300'

Then, we load the evaluation data:

In [ ]:
start = time.time()
evl_data = {}

for data, label, basename in evlkalaka:
        if basename not in evl_data:
                evl_data[basename] = {'data':[], 'label':label}
        evl_data[basename]['data'].append(data)

for basename in evl_data:
        evl_data[basename]['data'] = np.concatenate(evl_data[basename]['data'])

print(f'Finished reading all data in {time.time() - start}')

And apply the model(s) to the new `'evl'` data:

In [ ]:
start = time.time()

results_evl = {}
results_evl['ref'] =  None
results_evl['hyp'] =  np.empty(len(evl_data),dtype=np.int32)
results_evl['llhs'] = np.empty((len(evl_data), len(LANGUAGES)), dtype=np.float64)
results_evl['fileids'] = list()


# Obtain LLH matrix
for i, fileid in tqdm(enumerate(sorted(evl_data)), total=len(evl_data)):

    data = evl_data[fileid]['data']  # the features
    results_evl['fileids'].append(fileid)     #fileid

    # obtain the log-likelihood score for each model and store
    results_evl['llhs'][i,:] = np.array([models[lang].score(data) for lang in LANGUAGES])

    # Obtain the maximum likelihood languge estimation
    ix = np.argmax(results_evl['llhs'][i,:])
    results_evl['hyp'][i] = LANG2ID[LANGUAGES[ix]]    

print(f'Finished predicting all data in {time.time() - start}')

## Create the predictions file

The predictions file used for submission and scoring is a CSV file containing the predictions of both the `dev` and `evl` partitions.
The file has two fields: fileId and Lang. The fileId is the unique audio file identifier and the Lang field is the language prediction (numeric from 1 to 6). The predictions file name must be as follows:

`T<X>_G<YY>_<SYSTEMID>.csv` 

where `<X>` can be 1 or 2 depending on being a system for track 1 or track 2 evaluation; `<YY>` is the students' group number (use 2 digits) and `<SYSTEMID>` is a identifying string for that submission/system.

In [ ]:
group, system = '00', 'baseline_train100'
with open(f'{os.getcwd()}/T1_G{group}_{system}.csv', 'w') as file:
    csv_writer = csv.writer(file) # CSV writer
    csv_writer.writerow(('fileId', 'Lang')) # Header of the CSV

    # Save dev results
    for i in range(len(results_dev['fileids'])):
        csv_writer.writerow((results_dev['fileids'][i], results_dev['hyp'][i]))
    # Save evl results
    for i in range(len(results_evl['fileids'])):
        csv_writer.writerow((results_evl['fileids'][i], results_evl['hyp'][i]))

## Submit your prediction
You can submit your prediction in the following Kaggle competition: https://www.kaggle.com/competitions/speech-processing-lab-2/

## What should/can you do next?
**Everything!!** Try to extend and improve the feature extraction. Try to play with the parameters. Try completely different feature extraction modules (look for openSMILE, torchaudio,). Try to increase the model complexity. Try different modeling approaches. Try to understand the impact of the different modifications. Once you are happy with your system, try to train on the full set and check the impact of adding data to your system.


# PART 2 - Using pre-trained embeddings (Track 2)

There exist plenty of resources and pre-trained models that can be extremely useful for our task. For instance, x-vectors are currently the state of the art approach to obtain speech embeddings that characterize very efficiently speaker or language, among others. Particularly, the following x-vector model is available and it has been trained using a large corpus of 107 languages for language identification: https://huggingface.co/speechbrain/lang-id-voxlingua107-ecapa

You can obtain it from the `speechbrain` module, that you need to install now if you are using Google Colab:



In [ ]:
# If you are using Google Colab, you'll have to install the speechbrain module
raise CheckThisCell ## <---- Remove this after completing/checking this cell
!pip install speechbrain

The following code cell imports such model and shows how to obtain an embedding vector:

In [ ]:
import torchaudio
import speechbrain

from speechbrain.pretrained import EncoderClassifier
from torch.nn.utils.rnn import pad_sequence

language_id = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="tmp")

signal = language_id.load_audio(f'{CWD}/train100/audio/0a5c0729.wav')
emb =  language_id.encode_batch(signal)

In fact, the model is trained for language identification of 107 languages and we could use it directly for identification:

In [ ]:
prediction =  language_id.classify_batch(signal)

Invest some time to inspect the model and the outputs. Notice that the six target languages are included among the 107. The output indices are: 

```python
XVEC_LANG_INDEX = (24,13,20,29,75,22) 
```

corresponding respectively to the following languages:

```python 
LANGUAGES = ('Basque',  'Catalan',  'English',  'Galician',  'Portuguese',  'Spanish')
```

Knowing this, it should be easy to obtain the predicted class among the six possible candidates:

In [ ]:
XVEC_LANG_INDEX = (24,13,20,29,75,22) 

# Obtain the predicted class out of the 6 target ones
raise CheckThisCell ## <---- Remove this after completeing/checking this cell

## Using the x-vector up-stream model for language ID

Let's try first something simple: use the the pre-trained model for idenfication. In this simple test, you don't need to train anyhing simply classify the dev and test sets. To do so, first configure the right transformation for the Kalaka class (one that simply loads the audio without any chunking):

In [ ]:
# Confifure the tranformation
raise CheckThisCell ## <---- Remove this after completeing/checking this cell

transform = { 
                'raw_xvec' :
                {
                    'audio_transform': None, ## <--- You need to modify this here
                    'chunk_transform': None,
                    'chunk_size': 0,
                    'chunk_hop':0   
                }
            }

# Download and feature extract
transform_id = 'raw_xvec'


devkalaka = Kalaka(CWD, 'dev', 
                 transform_id=transform_id, 
                 audio_transform=transform[transform_id]['audio_transform'], 
                 chunk_transform=transform[transform_id]['chunk_transform'],
                 chunk_size=transform[transform_id]['chunk_size'], 
                 chunk_hop=transform[transform_id]['chunk_hop']
                )

Now you have all the elements to predict langauge using the langID pre-trained model.
In case you have access to a GPU, it may be good using a Pytorch Dataloader to batch the dev samples:

In [ ]:
import torch
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

Since the audios are of different size, we need to pass an auxiliary function to the dataloader that handles this (by adding 0s). This can be done as follows:

In [ ]:
def collate_batch(batch): 
    label_list, audio_list, basename_list, audiolen_list = [], [], [], []

    for (_audio,_label, _basename) in batch:
        label_list.append(_label)
        audio_list.append(_audio)
        basename_list.append(_basename)
        audiolen_list.append(_audio.shape[0])

    label_list = torch.tensor(label_list, dtype=torch.int64)
    audio_list = pad_sequence(audio_list, batch_first=True, padding_value=0)
    max_len = max(audiolen_list)
    audiolen_list = torch.tensor([l/max_len for l in audiolen_list])

    return audio_list.to(device),label_list.to(device), basename_list, audiolen_list.to(device)

batch_size = 10
dataloader = torch.utils.data.DataLoader(
        dataset=devkalaka,
        batch_size=batch_size,
        collate_fn=collate_batch,
        shuffle=False  # <-- We want to keep the original order of the dev set
)


Complete the following piece of code to store properly the hypothesis and the reference:

In [ ]:
import time
from tqdm import tqdm
start = time.time()

results_dev = {}
results_dev['ref'] =  []
results_dev['hyp'] =  []
results_dev['fileids'] = [] 

for i, batch in tqdm(enumerate(dataloader), total=1+len(devkalaka)//batch_size):
    data, label, basename, audiolen = batch
    predictions = language_id.classify_batch(data, wav_lens=audiolen)
    
    # Complete the code to store the hypothesis (careful if you use argmax, you will ned to add 1 to the predicted class),
    # the reference and the fileids. In the call to the prediction method, you should pass the audio length information, so that 
    # the padded 0s can be ignored
    raise CheckThisCell ## <---- Remove this after completeing/checking this cell
    
print(f'Finished preocessing all data in {time.time() - start}')

And now let's see how well this model behaves on our dev partition:

In [ ]:
ref, hyp = results_dev['ref'], results_dev['hyp']

conf_matrix = confusion_matrix(ref, hyp)

fig, ax = plt.subplots(figsize=(3.0, 3.0))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center')

plt.xlabel('Predictions', fontsize=12)
plt.ylabel('Actuals', fontsize=12)
plt.title(f'Confusion Matrix\n(Accuracy {100*accuracy_score(ref, hyp):.2f})', fontsize=12)
plt.show()

That was pretty impressive, right!?! And  easier than Part I of this lab! 

Well, here comes some bad news: you can not use this langid classifier in the challenge. However, you can use the pre-trained x-vector embeddings as a feature extractor to train your language classification system (may be a simple K-means on top of the x-vectors work well). You can also try to change the classification head of the x-vector model and fine-tune with the challenge data. You have plenty of options, but remember, any system that use these pre-trained embeddings (or any similar ones) are only valid for the Track 2 of the Challenge and the prediction file should be  namedlike this: `T2_G<YY>_<SYSTEMID>.csv` 



# What should you deliver at the end of this lab assignment?
You should deliver the following three elements:
- You must submit at least one prediction file to the Kaggle competition in the format previously described: https://www.kaggle.com/competitions/speech-processing-lab-2/
- You must submit (via Fênix) the modified notebook (or code) of your proposed systems(s).
- You must submit a report (via Fênix) of maximum 2 pages describing your system, approaches (may be unsuccesful), lessons learnt, results on the dev partition, etc. You can use the following Overleaf template for the report: https://www.overleaf.com/latex/templates/interspeech-2023-paper-kit/kzcdqdmkqvbr

# Contacts and support
You can contact the professors during the classes or the office hours.

Particularly, for this second laboratory assignment, you should contact Prof. Alberto Abad: alberto.abad@tecnico.ulisboa.pt



